In [1]:
import torch
from transformers import AutoTokenizer, AutoModel,BertConfig
import numpy as np
from tqdm import tqdm
import h5py 
import os
import glob
import utils
from torchinfo import summary
os.environ['CUDA_VISIBLE_DEVICES']='2'

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True).to('cuda')
model.eval();

/home/ztang/.conda/envs/torch_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/ztang/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertM

## Lenti-MPRA

In [ ]:
#Dataset for representation extraction
celltype = 'HepG2'
file = h5py.File('../../data/lenti_MPRA/'+celltype+'_data.h5','r')
file.keys()

In [7]:
#Max size from BPE tokenizer
batch_size = 128
max_len = 0
for dataset in ['seq_train','seq_valid','seq_test']:
    for seq in file[dataset]:
        seq = seq.decode()
        input_ids = tokenizer([seq])["input_ids"]
        if len(input_ids[0]) > max_len:
            max_len = len(input_ids[0])
print(max_len)

60


In [ ]:
#output file location
hidden_output = h5py.File('../../data/lenti_MPRA_embed/dnabert_'+celltype+'.h5','w')

batch_size = 128

for dataset in ['seq_train','seq_valid','seq_test']:
    output_cache = []
    for i in tqdm(range(0,len(file[dataset]),batch_size)):
        seq = file[dataset][i:i+batch_size].astype('U230')
        token = tokenizer(seq.tolist(), return_tensors="pt",padding='max_length',max_length=59)
        input_ids = token["input_ids"].to('cuda')
        attention_mask = token['attention_mask'].to('cuda')
        hidden_states = model(input_ids,attention_mask)[0].cpu().detach().numpy() # [1, sequence_length, 768]
        output_cache.extend(hidden_states)
    hidden_output.create_dataset(name='x'+dataset[3:],data = np.array(output_cache))
    hidden_output.create_dataset(name='y'+dataset[3:],data = file['y'+dataset[3:]][:])
hidden_output.close()

## Random Weight Intilization

In [2]:
mean = []
for n,v in model.named_parameters():
    mean.append(np.mean(v.detach().cpu().numpy()))
print(np.mean(mean))
print(np.std(mean))

0.048895333
0.122680694


In [3]:
# Randomly initialize weight in DNABERT model
for m in model.modules():
    m.apply(model._init_weights)

In [4]:
mean = []
for n,v in model.named_parameters():
    mean.append(np.mean(v.detach().cpu().numpy()))
print(np.mean(mean))
print(np.std(mean))

0.18116103
0.38514945


In [7]:
#Dataset for representation extraction
celltype = 'HepG2'
file = h5py.File('../../data/lenti_MPRA/'+celltype+'_data.h5','r')
file.keys()

<KeysViewHDF5 ['onehot_test', 'onehot_train', 'onehot_valid', 'seq_test', 'seq_train', 'seq_valid', 'y_test', 'y_train', 'y_valid']>

In [8]:
#output file location
hidden_output = h5py.File('../../data/lenti_MPRA_embed/randbert_'+celltype+'.h5','w')
batch_size = 128

for dataset in ['seq_train','seq_valid','seq_test']:
    output_cache = []
    for i in tqdm(range(0,len(file[dataset]),batch_size)):
        seq = file[dataset][i:i+batch_size].astype('U230')
        token = tokenizer(seq.tolist(), return_tensors="pt",padding='max_length',max_length=59)
        input_ids = token["input_ids"].to('cuda')
        attention_mask = token['attention_mask'].to('cuda')
        hidden_states = model(input_ids,attention_mask)[0].cpu().detach().numpy() # [1, sequence_length, 768]
        output_cache.extend(hidden_states)
    hidden_output.create_dataset(name='x'+dataset[3:],data = np.array(output_cache))
    hidden_output.create_dataset(name='y'+dataset[3:],data = file['y'+dataset[3:]][:])
hidden_output.close()

100%|██████████| 193/193 [00:25<00:00,  7.72it/s]


## Chip/Clip seq data

In [ ]:
max_len = 0
file_list = glob.glob('../data/chip/*.h5')
for file in file_list:
    tf_name = file.split('/')[-1][:-7]
    bert_output = h5py.File('../data/chip/DNABERT/'+tf_name+'_200.h5','w')
    batch_size = 128
    file = h5py.File(file,'r')
    for label in ('train','valid','test'):
        output_cache = []  
        for i in tqdm(range(0,len(file['x_'+label]),batch_size)):
            seq = file['x_'+label][i:i+batch_size].astype('int')
            seq = np.transpose(seq,(0,2,1))
            seq = utils.onehot_to_seq(seq)
            input_ids = tokenizer(seq, return_tensors="pt",padding='max_length',max_length=57)["input_ids"].to('cuda')
            hidden_states = model(input_ids)[0].cpu().detach().numpy() # [1, sequence_length, 768]
            output_cache.extend(hidden_states)
        bert_output.create_dataset(name='x_'+label,data = np.array(output_cache),dtype = 'float32')
        bert_output.create_dataset(name='y_'+label,data = file['y_'+label][:],dtype='int') 
    bert_output.close()


In [ ]:
max_len = 0
file_list = glob.glob('../data/eclip/*.h5')
for file in file_list:
    tf_name = file.split('/')[-1][:-7]
    bert_output = h5py.File('../data/eclip/DNABERT/'+tf_name+'_200.h5','w')
    batch_size = 128
    file = h5py.File(file,'r')
    for label in ('train','valid','test'):
        output_cache = []  
        for i in tqdm(range(0,len(file['X_'+label]),batch_size)):
            seq = file['X_'+label][i:i+batch_size].astype('int')
            seq = np.transpose(seq,(0,2,1))
            seq = utils.onehot_to_seq(seq)
            input_ids = tokenizer(seq, return_tensors="pt",padding='max_length',max_length=53)["input_ids"].to('cuda')
            hidden_states = model(input_ids)[0].cpu().detach().numpy() # [1, sequence_length, 768]
            output_cache.extend(hidden_states)
        bert_output.create_dataset(name='x_'+label,data = np.array(output_cache),dtype = 'float32')
        bert_output.create_dataset(name='y_'+label,data = file['Y_'+label][:],dtype='int') 
    bert_output.close()


## MT Splice data

In [ ]:
file = h5py.File('../data/alternative_splicing/delta_logit.h5','r')
bert_output = h5py.File('../data/alternative_splicing/DNABERT_splice.h5','w')
batch_size = 32
max_len = 0
for label in ('valid','test','train'):
    l_cache = []
    r_cache = [] 
    for i in tqdm(range(0,len(file['x_'+label]),batch_size)):
        seq = file['x_'+label][i:i+batch_size].astype('int')
        seq = utils.onehot_to_seq(seq)
        clean_seq = seq
        #clean_seq = [s if 'N' not in s else s.replace('N','[PAD]') for s in seq ]
        l_seq = []
        r_seq = []
        for seq in clean_seq:
            l_seq.append(seq[:400])
            r_seq.append(seq[400:])
        l_input = tokenizer(l_seq, return_tensors="pt",padding='max_length',max_length=270)["input_ids"].to('cuda')
        r_input = tokenizer(r_seq, return_tensors="pt",padding='max_length',max_length=270)["input_ids"].to('cuda')

        l_output = model(l_input)[0].cpu().detach().numpy() # [1, sequence_length, 768]
        r_output = model(r_input)[0].cpu().detach().numpy() # [1, sequence_length, 768]
        l_cache.extend(l_output)
        r_cache.extend(r_output)
    bert_output.create_dataset(name='xl_'+label,data = np.array(l_cache),dtype = 'float32')
    bert_output.create_dataset(name='xr_'+label,data = np.array(r_cache),dtype = 'float32')
    bert_output.create_dataset(name='y_'+label,data = file['y_'+label][:],dtype='float32') 


In [ ]:
for key in bert_output.keys():
    print(bert_output[key].shape)

bert_output.close()

## RNA-enlong data

In [ ]:
bert_output = h5py.File('../data/RNAenlong/dnabert_embed.h5','w')
file = h5py.File('../data/RNAenlong/insert_dataset.h5','r')
batch_size = 32
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)
    token_seq = tokenizer(string_seq, return_tensors="pt",padding='max_length',max_length=46)["input_ids"].to('cuda')
    output_cache = []
    for seq_i in tqdm(range(0,len(token_seq),batch_size)):
        hidden_states = model(token_seq[seq_i:seq_i+batch_size])[0].cpu().detach().numpy() # [1, sequence_length, 768]
        output_cache.extend(hidden_states)
    bert_output.create_dataset(name=key,data = np.array(output_cache))
    bert_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

In [ ]:
for key in bert_output.keys():
    print(bert_output[key].shape)

bert_output.close()